In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json 
import cv2
import mediapipe as mp

In [2]:
import time

In [3]:
%matplotlib inline

### Get Keypoints from a frame

In [4]:
mp_holistic = mp.solutions.holistic # human pose, face landmarks, and hand tracking 
mp_drawing = mp.solutions.drawing_utils # To draw the keypoints

In [5]:
def mp_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # By deafult captured image isn't rgb
    image.flags.writeable = False # Can't edit during processing
    results = model.process(image)
    image.flags.writeable = True 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,      
    mp_drawing.DrawingSpec(
        color=(255,0,255),
        thickness=1,
        circle_radius=1
      ),
      mp_drawing.DrawingSpec(
        color=(0,255,255),
        thickness=1,
        circle_radius=1
      ))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [7]:
# cap = cv2.VideoCapture(0)
# holistic = mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5)
# while holistic:
#     while cap.isOpened():
        
#         ret, frame = cap.read()
#         print(ret)
        
#         image, results = mp_detection(frame, holistic)
        
#         draw_landmarks(image, results)
        
#         cv2.imshow("OpenCV Feed", image)
        
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
        
#     cap.release()
#     cv2.destroyAllWindows()

In [7]:
with open("WLASL_v0.3.json") as f:
    data = json.loads(f.read())

In [8]:
def get_video_ids(word):
    results = []
    for i in data:
        if(i['gloss'] == word):
            for j in i['instances']:
                results.append(j['video_id'])
    return results

In [9]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    #return np.concatenate([pose, face, lh, rh])
    return np.concatenate([pose, lh, rh])

In [11]:
# data[0]['gloss']

In [12]:
# final_res = {}
# for i in data:
#     final_res[i['gloss']] = len(i['instances'])
       

In [13]:
# final_res["there"]

In [1]:
# for i in final_res:
#     if(final_res[i] > 15):
#         print('"', i, '"', end = ',', sep = '')

In [15]:
# import copy
# results = get_video_ids("hello")
# print(results)
# final_result = []

# for result in results:
#     holistic = mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5)
#     im_result = None
#     while holistic:
#         cap = cv2.VideoCapture(f"videos/{result}.mp4")
#         print("----------------------------------------------------------------------------------------------------")
#         while cap.isOpened():

#             ret, frame = cap.read()
#             if(not(ret)):
#                 break
#             image, im_result = mp_detection(frame, holistic)
#             #print(im_result.pose_landmarks)
#             draw_landmarks(image, im_result)
            
#             cv2.imshow("OpenCV Feed", image)

#             if cv2.waitKey(10) & 0xFF == ord('q'):
#                 break
        
#         if(im_result is not None and im_result.face_landmarks is not None):
#             final_result.append(copy.deepcopy(im_result))
#         print(final_result)
#         cap.release()
#         cv2.destroyAllWindows()
#         break

In [2]:
# len(get_video_ids("brother"))

In [10]:
# import os
# import sys
words = np.array(["book","drink","computer","before","chair","go","clothes","who","candy","cousin","deaf","fine","help","no","thin"])
# min_frame_count = sys.maxsize
# min_count = 1000000000
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     # NEW LOOP
#     # Loop through actions
    
#     for word in words:
#         videos = get_video_ids(word)
#         # Loop through sequences aka videos
#         count = 0
#         for vid_id in videos:
#             # Loop through video length aka sequence length
#             if(os.path.isfile(f"videos/{vid_id}.mp4")):
#                 count+= 1
#                 cap = cv2.VideoCapture(f"videos/{vid_id}.mp4")
#                 length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#                 if(length < min_frame_count):
#                     #print(vid_id)
#                     #print(min_frame_count)
#                     min_frame_count = length
#                 cap.release()
#                 cv2.destroyAllWindows()
#             #print(count)
#         if(count < min_count):
#             min_count = count
# # print(min_frame_count)
# # print(min_count)
# #Set mediapipe model 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     # NEW LOOP
#     # Loop through actions
#     for word in words:
#         videos = get_video_ids(word)
#         # Loop through sequences aka videos
#         for vid_id in videos:
#             # Loop through video length aka sequence length
#             cap = cv2.VideoCapture(f"videos/{vid_id}.mp4")
#             count = 0
#             cur_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#             curr_skip_len = cur_len//min_frame_count
#             fram_no = 0
#             while cap.isOpened():
#                 # Read feed
#                 ret, frame = cap.read()
#                 if(not(ret)):
#                     break
#                 # Make detections
#                 image, results = mp_detection(frame, holistic)
#                 # print(results)

#                 # Draw landmarks
#                 draw_landmarks(image, results)
                
#                 cv2.imshow("OpenCV Feed", image)

#                 # NEW Export keypoints
#                 keypoints = extract_keypoints(results)
#                 if(not(os.path.isdir(os.path.join("Data1", word, vid_id)))):
#                     os.makedirs(os.path.join("Data1", word, vid_id))
#                 #file = open(os.path.join("Data", word, vid_id, str(count)+".npy"), 'x')
#                 #file.close()
#                 npy_path = os.path.join("Data1", word, vid_id, str(count))
                
                
#                 if(not(os.path.isfile(npy_path))):
#                     np.save(npy_path, keypoints)
#                 fram_no += curr_skip_len
#                 count+=1
#                 #print(count, min_frame_count)
#                 if(count+1 >= min_frame_count):
#                     break
#                 currentPos = cap.get(cv2.CAP_PROP_POS_FRAMES)
#                 #print("Current", currentPos)
#                 #print("Supposed", fram_no)
#                 cap.set(cv2.CAP_PROP_POS_FRAMES, fram_no)
#                 # Break gracefully
#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break
#             #print("Count", count)
#             #if(os.path.isdir(os.path.join("Data", word, vid_id))):
#                 #print(len(os.listdir(os.path.join("Data", word, vid_id))))
#             cap.release()
#             cv2.destroyAllWindows()

##  Preprocess Data and Create Labels and Features

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(words)}

In [13]:
label_map

{'book': 0,
 'drink': 1,
 'computer': 2,
 'before': 3,
 'chair': 4,
 'go': 5,
 'clothes': 6,
 'who': 7,
 'candy': 8,
 'cousin': 9,
 'deaf': 10,
 'fine': 11,
 'help': 12,
 'no': 13,
 'thin': 14}

In [14]:
import os
sequences, labels = [], []
for word in words:
    videos = get_video_ids(word)
    # Loop through sequences aka videos
    for vid_id in videos:
        window = []
        if(os.path.isdir(os.path.join("Data_Without_face", word, vid_id))):
            if(len(os.listdir(os.path.join("Data_Without_face", word, vid_id))) + 1 <= 29):
                print(vid_id)
            for frame_num in range(len(os.listdir(os.path.join("Data_Without_face", word, vid_id)))):
                res = np.load(os.path.join("Data_Without_face", word, vid_id, "{}.npy").format(frame_num))
                window.append(res)
                print(len(res))
            sequences.append(window)
            #print(len(window))
            labels.append(label_map[word])

    # data augmentation        
    for vid_id in videos:
        window = []
        if(os.path.isdir(os.path.join("Data_Without_face", word, vid_id))):
            if(len(os.listdir(os.path.join("Data_Without_face", word, vid_id))) + 1 <= 29):
                print(vid_id)
            for frame_num in range(len(os.listdir(os.path.join("Data_Without_face", word, vid_id)))):
                res = np.load(os.path.join("Data_Without_face", word, vid_id, "{}.npy").format(frame_num))
                res/=1.5
                window.append(res)
                print(len(res))
            sequences.append(window)
            #print(len(window))
            labels.append(label_map[word])

69241
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
07069
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
07068
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
07070
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
07099
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
07074
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
69241
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
07069
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
07068
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
07070
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
07099
258
258
258
25

258
258
258
258
258
258
258
258
05730
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
65167
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
05731
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
05732
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
05733
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
05734
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
05727
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
05739
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
05740
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
05741
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258


258
258
258
258
258
258
258
258
258
258
63225
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
63236
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
63237
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
08929
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
08916
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
08917
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
08918
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
08919
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
08920
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
08921
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258


258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
21870
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
21890
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
21891
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
21871
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
21872
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
21869
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
21874
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
21878
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
21883
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
258
69364
258
258
258
258
258
258
258
258


In [15]:
for i in sequences:
    print(len(i))

23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
2

In [16]:
np.array(sequences).shape

(360, 23, 258)

In [17]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10

In [18]:
X= np.asarray(sequences)

In [19]:
X.shape

(360, 23, 258)

In [20]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.15, random_state = 42)

### Build and Train LSTM Neural Network

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Bidirectional, Embedding
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(GRU(64, return_sequences=True, activation='relu', input_shape=(23,258)))
model.add(GRU(128, return_sequences=True, activation='relu'))
model.add(GRU(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(words.shape[0], activation='softmax'))

# model = Sequential()
# # model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(23,1662)))
# # model.add(LSTM(128, return_sequences=True, activation='relu'))
# # model.add(LSTM(64, return_sequences=True, activation='relu'))
# # model.add(LSTM(32, return_sequences=False, activation='relu'))
# # # model.add(Dense(64, activation='relu'))
# # model.add(Dense(32, activation='relu'))
# # model.add(Dense(words.shape[0], activation='softmax'))

# model.add(LSTM(128, return_sequences=False, activation='relu', input_shape=(23,1662)))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(words.shape[0], activation='softmax'))

In [23]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
10/10 [==============================] - 4s 38ms/step - loss: 2.6906 - categorical_accuracy: 0.0784
Epoch 2/500
10/10 [==============================] - 0s 36ms/step - loss: 2.6300 - categorical_accuracy: 0.1340
Epoch 3/500
10/10 [==============================] - 0s 37ms/step - loss: 2.5134 - categorical_accuracy: 0.1307
Epoch 4/500
10/10 [==============================] - 1s 64ms/step - loss: 2.4249 - categorical_accuracy: 0.1601
Epoch 5/500
10/10 [==============================] - 1s 67ms/step - loss: 2.3812 - categorical_accuracy: 0.1340
Epoch 6/500
10/10 [==============================] - 0s 45ms/step - loss: 2.3301 - categorical_accuracy: 0.1503
Epoch 7/500
10/10 [==============================] - 0s 35ms/step - loss: 2.3024 - categorical_accuracy: 0.1699
Epoch 8/500
10/10 [==============================] - 0s 35ms/step - loss: 2.2658 - categorical_accuracy: 0.1634
Epoch 9/500
10/10 [==============================] - 0s 35ms/step - loss: 2.2295 - categorical_accuracy:

In [ ]:
# model.summary()

In [26]:
import tensorflow as tf

In [27]:
yhat = model.predict(X_test)

yhat

array([[4.62170746e-10, 8.30354507e-10, 6.75313272e-09, 4.89721401e-08,
        1.56620450e-09, 1.51881780e-02, 4.58128739e-15, 6.76543221e-09,
        1.21608401e-09, 3.10892562e-11, 9.84811783e-01, 5.07597386e-09,
        2.82503302e-16, 3.29380327e-08, 2.14822493e-09],
       [9.92301166e-18, 1.61969012e-17, 6.47123039e-18, 4.72737781e-07,
        1.35534432e-22, 2.34781292e-11, 4.63140952e-29, 9.99943733e-01,
        5.95931020e-08, 2.76436013e-10, 1.34205657e-05, 4.22615303e-05,
        4.42678224e-28, 1.70586727e-08, 4.20766286e-12],
       [3.80542562e-14, 1.11592282e-15, 6.80486873e-15, 8.14405456e-03,
        2.08854354e-15, 1.81271744e-08, 3.32777821e-18, 1.32972591e-11,
        1.55925145e-02, 7.97297961e-09, 2.52014576e-10, 1.28207326e-01,
        8.73312568e-15, 8.31009328e-01, 1.70467645e-02],
       [1.46290024e-18, 1.22864448e-15, 4.45542407e-17, 1.31561877e-02,
        2.09978567e-21, 3.54573260e-08, 1.34347758e-26, 2.03588903e-02,
        9.66462255e-01, 5.22238752e-1

In [28]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [29]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

multilabel_confusion_matrix(ytrue, yhat)

array([[[25,  2],
        [ 0,  0]],

       [[22,  1],
        [ 3,  1]],

       [[25,  0],
        [ 1,  1]],

       [[24,  1],
        [ 2,  0]],

       [[22,  3],
        [ 1,  1]],

       [[23,  1],
        [ 3,  0]],

       [[26,  1],
        [ 0,  0]],

       [[24,  2],
        [ 1,  0]],

       [[20,  4],
        [ 3,  0]],

       [[24,  1],
        [ 2,  0]],

       [[23,  2],
        [ 2,  0]],

       [[23,  1],
        [ 3,  0]],

       [[22,  3],
        [ 1,  1]],

       [[26,  0],
        [ 0,  1]]], dtype=int64)

In [30]:
accuracy_score(ytrue, yhat)

0.18518518518518517

In [154]:
# checkpoint_path = "Checkpoints"
# model.save(checkpoint_path+"\\"+"LSTM_LLL_32_128_64_DD_64_32.h5")

In [187]:
# checkpoint_path = "Checkpoints"
# model = tf.keras.models.load_model(checkpoint_path + "\\" + "my_model_w_head_and_augment.h5")

In [188]:
# colors = [(245,117,16), (117,245,16), (16,117,245), (16,117,245)]
# def prob_viz(res, actions, input_frame, colors):
#     output_frame = input_frame.copy()
    
#     for num, prob in enumerate(res):
#         print(num, prob, colors)
#         cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
#         cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
#     return output_frame

In [72]:
# len(words)

In [1]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
try:
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        count = 0
        while cap.isOpened():
            done = False
            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mp_detection(frame, holistic)
            print(results)

            # Draw landmarks
            draw_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
    #         sequence.insert(0,keypoints)
    #         sequence = sequence[:30]
            sequence.append(keypoints)
            sequence = sequence[-23:]
            if len(sequence) == 23:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(words[np.argmax(res)])
                predictions.append(np.argmax(res))


            #3. Viz logic
                if np.unique(predictions[-20:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if words[np.argmax(res)] != sentence[-1]:
                                sentence.append(words[np.argmax(res)])
                        else:
                            sentence.append(words[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

                # Viz probabilities
                image = prob_viz(res, words, image, colors)
                    
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()
except Exception as e:
    print(e)
    cap.release()
    cv2.destroyAllWindows()

NameError: name 'cv2' is not defined